In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import sklearn
from simpletransformers.seq2seq import Seq2SeqModel, Seq2SeqArgs
import numpy as np

In [2]:
data_dir = 'data/'
df = pd.read_csv(data_dir+'gen_2_pairs.csv')
df["prefix"] = "nltocl"
df = df[["prefix", "input_text", "target_text"]]
q_train, q_test = train_test_split(df, random_state=10)


In [3]:
q_train.shape, q_test.shape

((322, 3), (108, 3))

In [4]:
q_train.head()

,prefix,input_text,target_text
27,nltocl,A store has 375 black shirts and 40 white shir...,A store has 375 black shirts and 40 white shir...
295,nltocl,An store has 52 cars. In each vehicle there ar...,An store has 52 cars. Each car has 4 doors. Ea...
116,nltocl,A man has a thousand ants in his stomach. He s...,A man has 1000 ants. He sells 213 ants. How ma...
105,nltocl,A dog has 2 birds. 1 bird flees from the dog. ...,A dog has 2 birds. 1 bird flees from the dog. ...
146,nltocl,Mom has ten cookies. Three cookies are given t...,A mother has 10 cookies. The mother gives 3 co...


In [5]:
# Model arguments
model_args = Seq2SeqArgs()

model_args.num_return_sequences = 1
model_args.max_length = 256
model_args.top_k = 50
model_args.top_p = 0.95

model_args.wandb_project = "NL to CL with BART"

In [6]:
prefix="nltocl:"
model = Seq2SeqModel(
    encoder_decoder_type="bart",
    encoder_decoder_name="/home/fabian/Documents/2021-2022/word-problem-solver/Experiments/models/20211207153056",
    args=model_args,
    use_cuda=True
)

In [7]:
model.predict(["The restaurant did had 175 normal chairs and 20 chairs for babies. How many does the restaurant have in total?"])

Generating outputs: 100%|██████████| 1/1 [00:00<00:00,  7.07it/s]


['A restaurant has 175 normal chairs and 20 chairs for babies. How many chairs does the restaurant have?']

In [9]:
sentences = ["The mistletoe will take 4 bolts. The dog eats a flower.",
"A dog takes 4 bolts. The of blue fiber and half that much white fiber. How many bolts in total does it take?",
"The boy has 2 apples. His girlfriend gets 2 apples. What is the amount of apples he has?",
"James decides to run 3 sprints 3 times a week.  He runs 60 meters each sprint.  How many total meters does he run a week?"
]
sentences = [prefix+s for s in sentences]
model.predict(sentences)

Generating outputs: 100%|██████████| 1/1 [00:00<00:00,  3.25it/s]


['A dog has 1 mistletoe. The dog eats 1 flower.',
 'A dog takes 4 bolts. The dog removes 1 fiber from every fiber and 1 fiber gets 1 bolt from every fibre. How many bolts does it take?',
 'A boy has 2 apples. A girlfriend gets 2 apples from a store. How many apples does the boy have?',
 'Anltocl:James runs 3 sprints 3 times a week.  He runs 60 meters each sprint. How many meters does he run a week?']

In [10]:
def avg_p(labels, preds):
    check = labels == preds
    invert = np.invert(check)
    return check.mean(), invert

def custom_eval(df, f=avg_p, print_errors=False):
    labels = np.array(df['target_text'].tolist())
    input_text = df['input_text'].tolist()
    preds = np.array(model.predict(input_text))
    input_text = np.array(input_text)
    score, mask = f(labels, preds)
    labels = labels[mask]
    preds = preds[mask]
    print(f'{f.__name__}: {score}')
    if print_errors:
        for l, p in zip(labels, preds):
            print(l)
            print(p)
            print()
    return score

custom_eval(q_test)


Generating outputs: 100%|██████████| 14/14 [00:05<00:00,  2.48it/s]

avg_p: 0.6111111111111112


0.6111111111111112

In [11]:
custom_eval(q_train)


Generating outputs: 100%|██████████| 41/41 [00:17<00:00,  2.38it/s]

avg_p: 0.7236024844720497


0.7236024844720497

In [13]:
labels = q_test['target_text'].tolist()
pdf = pd.DataFrame(zip(labels, labels), columns=['input_text', 'target_text'])
custom_eval(pdf)


Generating outputs: 100%|██████████| 14/14 [00:05<00:00,  2.45it/s]

avg_p: 0.6851851851851852


0.6851851851851852

In [ ]:
model.eval_model(q_train)
from os.path import basename

basename()